<a href="https://colab.research.google.com/github/khanyousefzai/IEEE_Kaggle_Chalenge/blob/master/IEEE_Kaggle_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
IEEE KAGGLE COMPETITION
IEEE Fraud Detection 2019
Prize: 20,000 Dollars
Date:09/05/2019
Author: 
Saad Hasan Khan Yousefzai
"""

'\nIEEE KAGGLE COMPETITION\nIEEE Fraud Detection 2019\nPrize: 20,000 Dollars\nDate:10/05/2019\nAuthor: \nSaad Hasan Khan Yousefzai\n'

In [1]:
#library imports
import numpy as np
import pandas as pd
import seaborn as sns
import keras
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
#import descartes
from keras.models import Sequential
from scipy.stats import chisquare, chi2_contingency, chi2
from sklearn import preprocessing
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import io
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from keras import regularizers
import folium
from folium.plugins import MarkerCluster
from sklearn.preprocessing import StandardScaler
import time
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

Using TensorFlow backend.


In [0]:
#Reading Data using pandas
train_identity = pd.read_csv('train_identity.csv')
train_transaction = pd.read_csv('train_transaction.csv')
test_identity = pd.read_csv('test_identity.csv')
test_transaction = pd.read_csv('test_transaction.csv')
sample_submision = pd.read_csv('sample_submission.csv')

In [3]:
%%time
#finding the dataset dimensions
print(train_transaction.shape, train_identity.shape)
print(test_transaction.shape, test_identity.shape)
#Merging the data set with transactions and identity in train and test.
#checking the time
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

(590540, 394) (144233, 41)
(506691, 393) (141907, 41)
CPU times: user 12 s, sys: 11.4 s, total: 23.4 s
Wall time: 23.5 s


In [4]:
%%time
#remove the extra data frame to save some memory
del train_transaction
del train_identity
del test_identity
del test_transaction

CPU times: user 43.3 ms, sys: 13.4 ms, total: 56.8 ms
Wall time: 56.7 ms


In [5]:
%%time
#Writing function to reduce memory
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [6]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage of properties dataframe is : 1959.8762512207031  MB
******************************
Column:  TransactionID
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  isFraud
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  TransactionDT
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  TransactionAmt
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  card1
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  card2
dtype before:  float64
dtype after:  uint16
******************************
******************************
Column:  card3
dtype before:  float64
dtype after:  uint8
******************************
******************************
Column:  card5
dtype before:  float64
d

In [14]:
"""
Checking differnce 
for i in train.columns:
    for j in test.columns:
        if(i == j):
            print(j)
        else:
            print(i)
"""
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,99,150,discover,142,credit,315,87,19,65535,NaN,NaN,1,1,0,0,0,1,0,0,1,0,2,0,1,1,14,65535,13,-123,65535,-84,65535,-1.0,-1.0,...,-101,0,-14,-29,-73,-101,-47,-101,-37,-101,89.0,NaN,9,-661,NaN,NaN,99,9,99,99,99,9,NaN,10,99,99,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404,150,mastercard,102,credit,325,87,65535,65535,gmail.com,NaN,1,1,0,0,0,1,0,0,0,0,1,0,1,1,0,65535,65535,0,65535,-84,65535,-1.0,-1.0,...,-101,0,-14,-29,-73,-101,-47,-101,-37,-101,89.0,NaN,9,-661,NaN,NaN,99,9,99,99,99,9,NaN,10,99,99,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490,150,visa,166,debit,330,87,287,65535,outlook.com,NaN,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,65535,65535,0,65535,-84,65535,-1.0,-1.0,...,-101,0,-14,-29,-73,-101,-47,-101,-37,-101,89.0,NaN,9,-661,NaN,NaN,99,9,99,99,99,9,NaN,10,99,99,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567,150,mastercard,117,debit,476,87,65535,65535,yahoo.com,NaN,2,5,0,0,0,4,0,0,1,0,1,0,25,1,112,112,0,94,0,-84,65535,-1.0,-1.0,...,-101,0,-14,-29,-73,-101,-47,-101,-37,-101,89.0,NaN,9,-661,NaN,NaN,99,9,99,99,99,9,NaN,10,99,99,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514,150,mastercard,102,credit,420,87,65535,65535,gmail.com,NaN,1,1,0,0,0,1,0,1,0,1,1,0,1,1,0,65535,65535,-123,65535,-84,65535,-1.0,-1.0,...,0,70787,-14,-29,-73,-101,-47,-101,-37,-101,100.0,NotFound,9,-480,New,NotFound,166,9,542,144,99,9,NaN,10,99,99,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [16]:
# Now we need to check the attribute the missing
# First calculate the columns which constitude missing ratio
# Then delete the columns which has more than 85 percent of the rows are empty
data_null = train.isnull().sum()/len(train) * 100
data_null = data_null.drop(data_null[data_null == 0].index).sort_values(ascending=False)[:500]
missing_data = pd.DataFrame({'Missing Ratio': data_null})
missing_data.head(433)
#Defining function to delete which has more than 85 percent null values
def missinghandler(asd):
    for i in asd.columns:
        if asd[i] >= asd[i].isnull().sum()/asd[0]
    

,Missing Ratio
id_27,99.124699
id_23,99.124699
id_33,87.589494
id_30,86.865411
id_34,86.824771
DeviceInfo,79.905510
id_16,78.098012
R_emaildomain,76.751617
id_31,76.245132
DeviceType,76.155722


590540